In [11]:
# modified from https://github.com/HarryVolek/PyTorch_Speaker_Verification
import os
import yaml

def load_hparam_str(hp_str):
    path = os.path.join('config', 'temp-restore.yaml')
    with open(path, 'w') as f:
        f.write(hp_str)
    return HParam(path)

def load_hparam(filename):
    stream = open(filename, 'r')
    docs = yaml.load_all(stream, Loader=yaml.FullLoader)
    hparam_dict = dict()
    for doc in docs:
        for k, v in doc.items():
            hparam_dict[k] = v
    return hparam_dict

def merge_dict(user, default):
    if isinstance(user, dict) and isinstance(default, dict):
        for k, v in default.items():
            if k not in user:
                user[k] = v
            else:
                user[k] = merge_dict(user[k], v)
    return user

class Dotdict(dict):
    """
    a dictionary that supports dot notation 
    as well as dictionary access notation 
    usage: d = DotDict() or d = DotDict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct=None):
        dct = dict() if not dct else dct
        for key, value in dct.items():
            if hasattr(value, 'keys'):
                value = Dotdict(value)
            self[key] = value

class HParam(Dotdict):

    def __init__(self, file_target, file_default=None):
        super(Dotdict, self).__init__()
        
        if file_default is not None : 
            hp_dict_base = load_hparam(file_default)
            hp_dict_update = load_hparam(file_target)
            hp_dict_base=merge_dict(hp_dict_update,hp_dict_base)
        else :
            hp_dict_base = load_hparam(file_target)
        
        hp_dotdict = Dotdict(hp_dict_base)
        for k, v in hp_dotdict.items():
            setattr(self, k, v)
            
    __getattr__ = Dotdict.__getitem__
    __setattr__ = Dotdict.__setitem__
    __delattr__ = Dotdict.__delitem__


In [12]:
print(HParam("default.yaml","v1.yaml"))
print(HParam("v1.yaml","default.yaml"))
print(HParam("default.yaml"))
print(HParam("v1.yaml"))

{'a': {'key': 'defaukt', 'val': 0}, 'b': {'key': 'b', 'val': 2}}
{'a': {'val': 3333, 'key': 'defaukt'}, 'b': {'key': 'bb', 'val': 22}}
{'a': {'key': 'defaukt', 'val': 0}, 'b': {'key': 'b', 'val': 2}}
{'a': {'val': 3333}, 'b': {'key': 'bb', 'val': 22}}


In [33]:
import torch
x = torch.rand(10,dtype=torch.double)
print(x)
x = torch.where(x < 0.5, x,x+11.)
print(x)

tensor([0.7178, 0.6077, 0.7774, 0.1240, 0.5061, 0.2580, 0.7909, 0.6329, 0.6704,
        0.1637], dtype=torch.float64)
tensor([11.7178, 11.6077, 11.7774,  0.1240, 11.5061,  0.2580, 11.7909, 11.6329,
        11.6704,  0.1637], dtype=torch.float64)


RuntimeError: expected scalar type float but found long int